<center><h1> Escuela Politecnica Nacional </h1> </center>

<center><h2> Data Mining & Machine Learning  </h2> </center>  



*Integrantes:*  

- Jose Jimenez  
- Samir Zurita  
- Bruce Soto

# 4.4.4. Regresión polinomial

Ahora podemos explorar estos conceptos de forma interactiva ajustando polinomios a los datos.

In [1]:
use strict;
use warnings;
use List::Util qw(min shuffle);
use Data::Dump qw(dump);
use constant np => 'numperl';
use constant False => 'False';
use constant True => 'True';
use constant None => 'None';
use AI::MXNet qw(mx);
use AI::MXNet qw(nd);
use d2l;
use d2l::Animator;
use d2l::Accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly qw(show_plot);

# 4.4.4.1. Generación del conjunto de datos
Primero necesitamos datos. Dado x
 , usaremos el siguiente polinomio cúbico para generar las etiquetas en los datos de entrenamiento y prueba:

$$ y=5+1.2x−3.4\frac{x^2}{2!}+5.6*\frac{x^3}{3!} + ϵ where ϵ∼N(0,0.1^2) 
$$
 
El término de ruido $ ϵ $
 sigue una distribución normal con media 0 y desviación estándar 0.1. Para la optimización, normalmente queremos evitar valores muy grandes de gradientes o pérdidas. Esta es la razón por la cual las características se reescalan de xi
 a xii!
 . Nos permite evitar valores muy grandes para exponentes grandes i
 . Sintetizaremos 100 muestras cada una para el conjunto de entrenamiento y el conjunto de prueba.

In [2]:
my $max_degree = 20; # Maximum degree of the polynomial
my $n_train = 100;
my $n_test = 100;
my $true_w = mx->nd->zeros([$max_degree]);
my $array = mx->nd->array([5, 1.2, -3.4, 5.6]);
$true_w->slice([0,3]) += $array->slice([0,3]);

#print(dump($true_w));

my $features = mx->nd->random_normal(shape=>[$n_train + $n_test, 1]);
#print(dump($features));

$features = mx->nd->shuffle($features);
my $aux = mx->nd->array([0 .. $max_degree - 1]);
my $poly_features = $features ** $aux->reshape([1,-1]);

for my $i (0 .. $max_degree - 1) {
    for my $j (0 .. $max_degree*10 - 1) {#$features->len
        $poly_features->[$j][$i] /= mx->nd->gamma(mx->nd->array([$i + 1]));
    }
}

my $labels = mx->nd->dot($poly_features, $true_w);
#labels += np.random.normal(scale=0.1, size=labels.shape)
$labels += mx->nd->random_normal(scale => 0.1, shape => $labels->shape);

<AI::MXNet::NDArray 200 @cpu(0)>

Nuevamente, los monomios almacenados en poly_featuresson reescalados por la función gamma, donde  $Γ(𝑛)=(𝑛−1)!$  . Eche un vistazo a las primeras 2 muestras del conjunto de datos generado. El valor 1 es técnicamente una característica, es decir, la característica constante correspondiente al sesgo.

In [3]:
print $features->slice([0,1])->aspdl, $poly_features->slice([0,1])->slice([0,1])->aspdl, $labels->slice([0,1])->aspdl;


[
 [1.50948]
 [1.96766]
]

[
 [      1 1.50948 2.27852 3.43936 5.19163 7.83664 11.8292 17.8559  26.953 40.6849 61.4129 92.7012  139.93 211.221 318.833 481.271 726.466 1096.58 1655.26 2498.58]
 [      1 1.96766 3.87169 7.61818   14.99 29.4952 58.0366 114.196   224.7 442.133 869.968  1711.8 3368.25 6627.57 13040.8 25659.9   50490 99347.2  195482  384642]
]
[18.3574 36.7292]

1

In [4]:
sub train_epoch_ch3{
    my ($net, $train_iter, $loss, $updater, $batch_size) = @_;
    my $metric = Accumulator->new(3);
    if ($updater->isa('Gluon::Trainer')){
        $updater->step($batch_size, 1);
    }  
    my ($X, $y, $y_hat, $l);
    while(defined(my $batch = <$train_iter>)){
        # Compute gradients and update parameters
        $X = $batch->[0];
        $y = $batch->[1]->astype('float32'); 
        autograd->record(sub {$y_hat = $net->($X);
                                $l = $loss->($y_hat, $y);
                                });
        $l->backward();
        if (ref($updater) eq 'AI::MXNet::Gluon::Trainer'){
            $updater->step($batch_size);
        }else{
            $updater->($batch_size);
        }
        $metric->add([ $l->sum->asscalar, accuracy($y_hat, $y), $y->size ]);
    }
    # Return training loss and training accuracy
    return ($metric->getitem(0) / $metric->getitem(2), $metric->getitem(1) / $metric->getitem(2));
}

# 4.4.4.2. Entrenamiento y prueba del modelo
Primero implementemos una función para evaluar la pérdida en un conjunto de datos dado.

In [5]:
sub evaluate_loss{
    my($net, $data_iter, $loss ) = @_;
    my $metric = Accumulator->new(2);
    my ($X , $y ,$l);
    while(defined(my $batch = <$data_iter>)){ 
        $X = $batch->[0];
        $y = $batch->[1]->astype('float32');
        my $l = $loss->($net->($X), $y);
        $metric->add([ $l->sum(), $l->size ]);
    }
    return ($metric->getitem(0) / $metric->getitem(1));
}

Ahora defina la función de entrenamiento.

In [6]:
sub train {
    my ($train_features, $test_features, $train_labels, $test_labels, $num_epochs) = @_;
    $num_epochs = 400;
    my $loss = mx->gluon->loss->L2Loss();
    my $net = mx->gluon->nn->Sequential();
    $net->add(mx->gluon->nn->Dense(1, use_bias => 0));
    $net->initialize();
    my $batch_size = min(10, ($train_labels->shape)[0]);
    
    my $train_iter = d2l->load_array([$train_features, $train_labels], $batch_size);
    my $test_iter = d2l->load_array([$test_features, $test_labels], $batch_size, 0);
    my $trainer = mx->gluon->Trainer($net->collect_params(), 'sgd', {'learning_rate' => 0.01});
    my $animator = Animator->new(xlabel => 'epoch', ylabel => 'loss', xlim => [1, $num_epochs], ylim => [1e-3, 1e2], legend => ['train', 'test']);
    
    for (my $epoch = 0; $epoch < $num_epochs; $epoch++) {
        #train_epoch_ch3($net, $train_iter, $loss, $trainer, $batch_size);
        if ($epoch == 0 || ($epoch + 1) % 20 == 0) {
            $animator->add($epoch + 1, [evaluate_loss($net, $train_iter, $loss), evaluate_loss($net, $test_iter, $loss)]);
        }
    }
    #show_plot($animator);
    print "weight:", $net->[0]->weight->data->aspdl, "\n";
}

# 4.4.4.3. Ajuste de función polinomial de tercer orden (normal)
Comenzaremos usando primero una función polinomial de tercer orden, que es del mismo orden que la función de generación de datos. Los resultados muestran que las pérdidas de entrenamiento y prueba de este modelo pueden reducirse de manera efectiva. Los parámetros del modelo aprendido también están cerca de los valores reales  $w=[5,1.2,−3.4,5.6]$
 .

In [7]:
train($poly_features->slice([0,$n_train-1],[0,3])->aspdl,
        $poly_features->slice([$n_train, -1],[0,3])->aspdl,
        $labels->slice([0,$n_train-1])->aspdl,
        $labels->slice([$n_train,-1])->aspdl
        );

weight:
[
 [  -0.067313 -0.00418149  -0.0277795   0.0302504]
]



1

# 4.4.4.4. Ajuste de función lineal (ajuste inferior)
Echemos otro vistazo al ajuste de funciones lineales. Después del declive en las primeras épocas, se vuelve difícil disminuir aún más la pérdida de entrenamiento de este modelo. Una vez completada la última iteración de época, la pérdida de entrenamiento sigue siendo alta. Cuando se utilizan para ajustar patrones no lineales (como aquí la función polinomial de tercer orden), los modelos lineales tienden a no ajustarse correctamente.

In [8]:
train($poly_features->slice([0,$n_train-1],[0,1])->aspdl,
        $poly_features->slice([$n_train, -1],[0,1])->aspdl,
        $labels->slice([0,$n_train-1])->aspdl,
        $labels->slice([$n_train,-1])->aspdl
        );

weight:
[
 [ 0.0224243 -0.0296813]
]



1

# 4.4.4.5. Ajuste de funciones polinómicas de orden superior (sobreajuste)

Ahora intentemos entrenar el modelo usando un polinomio de grado demasiado alto. Aquí, no hay datos suficientes para saber que los coeficientes de mayor grado deberían tener valores cercanos a cero. Como resultado, nuestro modelo excesivamente complejo es tan susceptible que está siendo influenciado por el ruido en los datos de entrenamiento. Aunque la pérdida de entrenamiento se puede reducir de manera efectiva, la pérdida de prueba sigue siendo mucho mayor. Muestra que el modelo complejo sobreajusta los datos.

In [9]:
train($poly_features->slice([0,$n_train-1],[0,-1])->aspdl,
        $poly_features->slice([$n_train, -1],[0,-1])->aspdl,
        $labels->slice([0,$n_train-1])->aspdl,
        $labels->slice([$n_train,-1])->aspdl,
        my $num_epochs=1500
        );

weight:
[
 [ -0.0293891  -0.0163153   0.0165222   0.0348838 -0.00997238   0.0529833  -0.0510336  -0.0555991  -0.0282405  -0.0570677  0.00979508  -0.0204335   0.0127222  0.00725428   0.0104055  -0.0652925   0.0214481   0.0656546   0.0212945  -0.0250604]
]



1

En las secciones subsiguientes, continuaremos analizando los problemas de sobreajuste y los métodos para tratarlos, como la disminución del peso y la deserción.

# 4.4.5. Resumen
* Dado que el error de generalización no se puede estimar en función del error de entrenamiento, simplemente minimizar el error de entrenamiento no significará necesariamente una reducción en el error de generalización. Los modelos de aprendizaje automático deben tener cuidado de protegerse contra el sobreajuste para minimizar el error de generalización.

* Se puede usar un conjunto de validación para la selección del modelo, siempre que no se use con demasiada liberalidad.

* Underfitting significa que un modelo no es capaz de reducir el error de entrenamiento. Cuando el error de entrenamiento es mucho menor que el error de validación, hay sobreajuste.

* Deberíamos elegir un modelo adecuadamente complejo y evitar usar muestras de entrenamiento insuficientes.

# 4.4.6. Ejercicios

*1.	¿Puedes resolver el problema de regresión polinomial exactamente? Pista: usa álgebra lineal.*  
Sí, el problema de regresión polinomial se puede resolver utilizando técnicas de álgebra lineal. La idea básica es crear una matriz de características polinómicas a partir de las variables independientes y luego resolver un sistema de ecuaciones lineales para encontrar los coeficientes del polinomio. Una vez que se encuentran los coeficientes, se pueden usar para hacer predicciones sobre la variable dependiente a partir de valores conocidos de la variable independiente.  Resolver el problema de regresión polinómica exacta puede dar como resultado un modelo complejo con una gran cantidad de coeficientes, lo que puede generar problemas de sobreajuste y generalización deficiente. Por lo tanto, es importante tener cuidado al elegir el grado correcto del polinomio y usar técnicas regulares para evitar el sobreajuste.  

*2.	Considere la selección de modelos para polinomios: Trazar la pérdida de entrenamiento frente a la complejidad del modelo (grado del polinomio). ¿Qué observas? ¿Qué grado de polinomio necesita para reducir la pérdida de entrenamiento a 0? Trace la pérdida de prueba en este caso. Genere la misma gráfica en función de la cantidad de datos.*

Graficar la pérdida de entrenamiento frente a la complejidad del modelo (orden polinomial) muestra que a medida que aumenta el orden del polinomio, la pérdida de entrenamiento disminuye hasta que alcanza un valor cercano a 0. Dado que los datos reales a menudo son ruidosos y no se ajustan perfectamente al modelo polinomial, podemos reducir la pérdida de entrenamiento a exactamente 0.  Para la pérdida de prueba, encontramos que inicialmente a medida que aumenta el grado del polinomio, la pérdida de prueba también disminuye hasta que se alcanza un mínimo. Sin embargo, a partir de ese momento, a medida que el orden del polinomio aumenta aún más, la pérdida de prueba comienza a aumentar, lo que indica que el modelo se está sobreajustando a los datos de entrenamiento.  Si se produce el mismo gráfico en función del volumen de datos, podemos ver que la pérdida de prueba generalmente disminuye a medida que aumenta el volumen de datos a medida que el modelo se ajusta mejor a los datos. Sin embargo, puede haber un punto en el que el modelo ya se ajuste a los datos lo suficientemente bien como para que aumentar la cantidad de datos no reduzca significativamente la pérdida de prueba.  En conclusión, estas gráficas lo ayudan a elegir el grado correcto de polinomio y la cantidad correcta de datos para su modelo de regresión polinomial. Esto se debe a que ayuda a visualizar el comportamiento de la pérdida y evitar el sobreajuste.  


*3.	¿Qué sucede si eliminas la normalización ( 1/i!) de las características del polinomio xi? ¿Puedes arreglar esto de alguna otra manera?*  

Si se elimina la normalización (1/i!) de las características del polinomio xi, los coeficientes del polinomio pueden volverse muy grandes, lo que puede hacer que el modelo sea sensible a las fluctuaciones en los datos de entrenamiento y albergar un alto riesgo de falla.  Para resolver este problema, una forma es utilizar la normalización de propiedades, ya que permite controlar la magnitud de los coeficientes del polinomio. Otra opción es usar técnicas regulares como la regularización L1 o L2, que penalizan los coeficientes grandes y ayudan a prevenir el sobreajuste. También se puede considerar una combinación de normalización y regularización para obtener mejores resultados.  

*4.	¿Puedes esperar ver cero errores de generalización?*  

Es poco probable que pueda esperar no ver ningún error de generalización. La generalización es el proceso de aplicar un modelo de trabajo a nuevos datos que no se usaron durante el entrenamiento. Incluso si el modelo se ajusta perfectamente a los datos de entrenamiento, es probable que todavía haya un pequeño error de generalización debido al ruido o la escasez de los datos reales.  Además, el sobreajuste es un fenómeno común en la regresión, donde el modelo se ajusta demasiado bien a los datos de entrenamiento, pero no se generaliza bien a los nuevos datos. Para evitar el sobreajuste, se pueden utilizar técnicas de regularización o se puede reducir la complejidad del modelo.  En resumen, no puede esperar no ver ningún error de generalización, pero se pueden tomar medidas para reducir los errores de generalización y mejorar la generalización del modelo.